In [1]:
import pandas as pd
import numpy as np
import sys
import os
import glob
from nexrad import get_lat_lon
import pvlib
import time as t
from util import *

In [2]:
def summarize_profile(infile, lat_lon):
    scan = pd.read_csv(infile)

    infile = infile.split('/')[-1]

    station = infile[:4]
    year = infile[4:8]
    month = infile[8:10]
    day = infile[10:12]
    hour = infile[13:15]
    minute = infile[15:17]
    second = infile[17:19]

    date = pd.to_datetime(f'{year}-{month}-{day} {hour}:{minute}:{second}Z')

    bin_widths = np.diff(scan['bin_lower'])
    if np.all(bin_widths == bin_widths[0]):
        bin_width = bin_widths[0]  # bin width in meters
    else:
        raise ValueError('Bin vertical widths are not consistent!')

    bin_width_km = bin_width / 1000  # bin width in km

    # Vertically integrated reflectivity (vir)
    #
    #  -- units in each elevation bin are reflectivity (cm^2/km^3)
    #
    #  -- multiply by height of each bin in km to get cm^2/km^2
    #
    #     == total scattering area in that elevation bin per 1 sq km.
    #        of area in x-y plane
    #
    #  -- add values together to get total scattering area in a column
    #     above a 1km patch on the ground (cm^2/km^2)
    #
    #  -- (NOTE: can multiply these values by 10^-8 to get units
    #      km^2/km^2, i.e., a unitless quantity. Interpretation: total
    #      fraction of a 1 square km patch on the ground that would be
    #      filled by the total scattering area of targets in the column
    #      above it. I.e., zap all the birds so they fall to the
    #      ground, and measure how much ground space is filled up.)

    linear_eta = np.array(scan['linear_eta'])

    # Compute vertically integrated reflectivity in cm2 / km2
    density = np.nansum(bin_width_km * linear_eta)
    density_unfiltered = np.nansum(bin_width_km * scan['linear_eta_unfiltered'])
    density_precip = density_unfiltered - density
    assert np.all(np.isnan(density_precip) | (density_precip >= 0))

    # Speed converted from m/s to km/h
    speed_km_h = scan['speed'] * (1/1000) * (3600/1)

    # Compute traffic rate in cm2 / km / h
    traffic_rate = np.nansum(bin_width_km * linear_eta * speed_km_h)
    traffic_rate_unfiltered = np.nansum(bin_width_km * scan['linear_eta_unfiltered'] * speed_km_h)
    traffic_rate_precip = traffic_rate_unfiltered - traffic_rate

    assert np.all(np.isnan(traffic_rate_precip) | (traffic_rate_precip >= 0))

    # Average velocity and speed weighted by reflectivity
    u = wtd_mean(linear_eta, scan['u'])
    v = wtd_mean(linear_eta, scan['v'])
    speed = wtd_mean(linear_eta, scan['speed'])

    # Average track as compass bearing (degrees clockwise from north)
    direction = pol2cmp(np.arctan2(v, u))

    # TODO: double check calculation
    percent_rain = (scan['percent_rain'] * scan['nbins']).sum() / scan['nbins'].sum()

    lat = lat_lon[station]["lat"]
    lon = lat_lon[station]["lon"]

    row = [station,
           lat,
           lon,
           date,
           density,
           density_precip,
           traffic_rate,
           traffic_rate_precip,
           u,
           v,
           speed,
           direction,
           percent_rain]

    return row

In [11]:
def get_day_info(station, year=None, start=None, end=None):
 
    if year is not None:
        # Get series of days from Jan 1 this year to Jan 1 next year
        start = pd.Timestamp(year=year, month=1, day=1)
        end = pd.Timestamp(year=year+1, month=1, day=1)
        
    elif start is None or end is None:
        raise ValueError('Either year or start/end must be set')
    
    days = pd.date_range(start, end, freq='D', tz='UTC')

    NEXRAD_LOCATIONS = nexrad.get_lat_lon()
    
    # location information
    lon = NEXRAD_LOCATIONS[station]['lon']
    lat = NEXRAD_LOCATIONS[station]['lat']
    loc = pvlib.location.Location(lat, lon)


    # Get sunrise, sunset, and transit times
    #   NOTE: methods other than 'spa' do not seem to correctly place
    #   sunset on the following date if it occurs after 00:00 UTC time
    day_info = loc.get_sun_rise_set_transit(days, method='spa')

    day_info['date'] = day_info.index.date
        
    # Add column for next sunrise, then drop last row, which corresponds to first day of following year
    day_info['next_sunrise'] = day_info['sunrise'].shift(-1)
    day_info = day_info.iloc[:-1]

    # Sanity checks
    assert(np.all(day_info['sunset'] > day_info['sunrise']))
    assert(np.all(day_info['next_sunrise'] > day_info['sunset']))
    
    return day_info

def get_period_info(station, start, end):
    
    info = get_day_info(station, start=start, end=end) 
    units = pd.Timedelta('1 hour')

    periods = []
    for day, row in info.iterrows():    
        periods.append([day.date(), 
                        'day', 
                        row['sunrise'], 
                        (row['sunset']-row['sunrise']) / units ])

        periods.append([day.date(),
                        'night', 
                        row['sunset'], 
                        (row['next_sunrise']-row['sunset']) / units ])

    periods = pd.DataFrame(periods, columns = ['period_date', 'period', 'period_start', 'period_length'])
    periods.index = periods['period_date'].astype(str) + '-' + periods['period']
        
    return periods, units

def add_period_info(df, station):
    
    start = df['date'].min().floor('1D') - pd.Timedelta('1 day')
    end = df['date'].max().ceil('D') + pd.Timedelta('1 day')
    
    # Get information about periods
    periods, units = get_period_info(station, start=start, end=end)
    
    # Cut df on period
    df['key'] = pd.cut(df['date'], periods['period_start'], labels=periods.index[:-1])
    
    # Join with period info
    df = df.join(periods, on='key')

    # Massage columns
    df['period_time'] = (df['date']-df['period_start']) / units
    df = df.drop(columns=['key', 'period_start'])

    return df

In [20]:
root = '../data'
lat_lon = get_lat_lon()

# TODO: could potentially be sped up by loading many profiles at once and using pandas commands, including
# TODO: group by to group by scan

station = 'KBOX'
year = 2017

file_list = f'{root}/file_lists/station_year_lists/{station}-{year}.txt'

with open(file_list, 'r') as infile:
    files = infile.read().split('\n')
    
files = [f'{root}/profiles/' + f for f in files]

outfile = f'{root}/scan_level/{station}_{year}.csv'

start = t.time()

column_names = ['station',                    # 0
                'lat',                        # 1
                'lon',                        # 2
                'date',                       # 3
                'density',                    # 4
                'density_precip',             # 5
                'traffic_rate',               # 6
                'traffic_rate_precip',        # 7
                'u',                          # 8
                'v',                          # 9
                'speed',                      # 10
                'direction',                  # 11
                'percent_rain']               # 12

# Get rows from individual files
i = 0
rows = []
for f in files:
    rows.append(summarize_profile(f, lat_lon))
    i += 1
#     if i > 500:
#         break

df = pd.DataFrame(rows, columns=column_names)

do_solar_elevation = True
if do_solar_elevation:
    # Add solar elevation (note: much faster to do in batch at end than row-by-row)
    solar_elev = pvlib.solarposition.spa_python(df['date'], df['lat'], df['lon'])
    df['solar_elevation'] = solar_elev['elevation'].values

    # Convert lat/lon to strings to preserve full precision --- other floats will be truncated
    df['lat'] = df['lat'].apply(str)
    df['lon'] = df['lon'].apply(str)

    # Add solar elevation to column names and write to file
    column_names.insert(4, 'solar_elevation')


df = add_period_info(df, station)    

df.to_csv(outfile, index=False, float_format='%.4f', date_format='%Y-%m-%d %H:%M:%SZ')

n_scans = len(rows)

elapsed = t.time()-start
print(f'  time={elapsed:.2f}, scans={n_scans}, per scan={elapsed/n_scans:.4f}')

  time=789.57, scans=74036, per scan=0.0107


In [16]:
display(df)

,station,lat,lon,date,density,density_precip,traffic_rate,traffic_rate_precip,u,v,speed,direction,percent_rain,solar_elevation,period_date,period,period_length,period_time
0,KBOX,41.95583,-71.1375,2016-12-01 00:03:12+00:00,1.89512,6128.05222,90.306328,294037.197067,-6.409912,8.432427,13.236677,322.759705,0.593118,-30.784972,2016-11-30,night,14.650450,2.814696
1,KBOX,41.95583,-71.1375,2016-12-01 00:09:01+00:00,1.56973,6285.13844,77.926838,301196.811940,-8.822204,1.418338,13.789852,279.133241,0.627265,-31.865643,2016-11-30,night,14.650450,2.911640
2,KBOX,41.95583,-71.1375,2016-12-01 00:14:52+00:00,1.07586,6483.90548,48.380454,312639.198866,-3.823556,5.297578,12.491416,324.179925,0.671157,-32.953046,2016-11-30,night,14.650450,3.009140
3,KBOX,41.95583,-71.1375,2016-12-01 00:20:41+00:00,0.98538,6356.35960,40.777783,306876.150583,-2.578335,5.923112,11.495222,336.476467,0.717018,-34.034492,2016-11-30,night,14.650450,3.106085
4,KBOX,41.95583,-71.1375,2016-12-01 00:26:30+00:00,1.38743,6233.14289,52.766460,303658.873173,-2.783096,6.054479,10.564389,335.312917,0.748635,-35.115867,2016-11-30,night,14.650450,3.203029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,KBOX,41.95583,-71.1375,2016-12-03 17:37:34+00:00,20.62141,0.13133,2224.005446,8.935903,-3.318587,-29.638959,29.958150,186.388630,0.010131,24.186874,2016-12-03,day,9.302935,5.702807
497,KBOX,41.95583,-71.1375,2016-12-03 17:47:19+00:00,19.10623,0.20225,2543.661373,7.631974,27.965166,-24.181704,36.981268,130.850235,0.008999,23.650841,2016-12-03,day,9.302935,5.865307
498,KBOX,41.95583,-71.1375,2016-12-03 17:57:03+00:00,15.90633,0.01318,2262.012329,0.679851,19.706371,34.168075,39.502309,29.974116,0.003743,23.044425,2016-12-03,day,9.302935,6.027529
499,KBOX,41.95583,-71.1375,2016-12-03 18:06:49+00:00,14.52861,0.01410,1395.471620,0.621859,5.382802,-25.965062,26.680529,168.287948,0.004639,22.366326,2016-12-03,day,9.302935,6.190307
